In [2]:
import torch
from torch import _dynamo
from torch._dynamo import config
import logging

In [3]:
config.log_level = logging.DEBUG
config.output_code = True

## Python constant in control-flow

Some if-else conditions is **constant** in a JIT module, such control-flow will be removed during the optimization.

In [4]:
def some_kernel(x):
    y = x
    if isinstance(x, torch.Tensor):
        y = x + 1
    return y * 2

In [5]:
mod = torch.compile(some_kernel)

In [6]:
x = torch.ones(10)
mod(x)

[2023-03-16 17:07:36,967] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /usr/lib/python3.8/contextlib.py
[2023-03-16 17:07:36,968] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /usr/lib/python3.8/contextlib.py
[2023-03-16 17:07:36,969] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /usr/lib/python3.8/contextlib.py
[2023-03-16 17:07:36,969] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /usr/lib/python3.8/contextlib.py
[2023-03-16 17:07:36,969] torch._dynamo.eval_frame: [DEBUG] skipping enable_dynamic /home/chunwei/trienv/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py
[2023-03-16 17:07:37,021] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing some_kernel
[2023-03-16 17:07:37,023] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/2329204500.py:2
[2023-03-16 17:07:37,024] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x []
[2023-03-16 17:07:37,024] torch._dynamo.symbolic_convert: [DEBUG] TRACE S

tensor([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

## Variable in a control flow would force a graph break

In [7]:
def some_kernel(x: torch.Tensor):
    if x.sum() > 0: # this should be constant for a JIT module
        return -x

In [8]:
mod = torch.compile(some_kernel)
mod(x)

[2023-03-16 17:07:40,934] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing some_kernel
[2023-03-16 17:07:40,934] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/3944027839.py:2
[2023-03-16 17:07:40,935] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x []
[2023-03-16 17:07:40,936] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR sum [TensorVariable()]
[2023-03-16 17:07:40,937] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 0 [GetAttrVariable(TensorVariable(), sum)]
[2023-03-16 17:07:40,939] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CONST 0 [TensorVariable()]
[2023-03-16 17:07:40,940] torch._dynamo.symbolic_convert: [DEBUG] TRACE COMPARE_OP > [TensorVariable(), ConstantVariable(int)]
[2023-03-16 17:07:40,942] torch._dynamo.symbolic_convert: [DEBUG] TRACE POP_JUMP_IF_FALSE 18 [TensorVariable()]
[2023-03-16 17:07:40,943] torch._dynamo.symbolic_convert: [DEBUG] generic_jump triggered compile
[2023-

__resume_at_12_2:
  2           0 JUMP_ABSOLUTE           14
              2 LOAD_FAST                0 (x)
              4 LOAD_ATTR                0 (sum)
              6 CALL_FUNCTION            0
              8 LOAD_CONST               1 (0)
             10 COMPARE_OP               4 (>)
             12 POP_JUMP_IF_FALSE       20

  3     >>   14 LOAD_FAST                0 (x)
             16 UNARY_NEGATIVE
             18 RETURN_VALUE
        >>   20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
__resume_at_18_3:
  2           0 JUMP_ABSOLUTE           20
              2 LOAD_FAST                0 (x)
              4 LOAD_ATTR                0 (sum)
              6 CALL_FUNCTION            0
              8 LOAD_CONST               1 (0)
             10 COMPARE_OP               4 (>)
             12 POP_JUMP_IF_FALSE       20
             14 LOAD_FAST                0 (x)
             16 UNARY_NEGATIVE
             18 RETURN_VALUE

  3     >>   20 LOAD_CONST     

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

It shows that the `POP_JUMP_IF_FALSE` cannot be pruned for we need the runtime value of the `x.sum()`, which is only accessible after it is executed.

# Function call and Frame

In Python VM, a function call will generate a new  frame, but this is not always true in Dynamo. 
Dynamo could alter the Code Object to inline some function call if necessary.

In [9]:
def third_fn(x: torch.Tensor):
    return x + 1

def main_fn(x: torch.Tensor, y:torch.Tensor):
    tmp = third_fn(x) # inline call here?
    return y + tmp

In [10]:
_dynamo.reset()
main_fn_ = torch.compile(main_fn)

For small function call, Dynamo will inline it directly, and that would save a frame.

In [11]:
main_fn_(x,x)

[2023-03-16 17:07:41,080] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn
[2023-03-16 17:07:41,081] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/285673021.py:5
[2023-03-16 17:07:41,081] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL third_fn []
[2023-03-16 17:07:41,083] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:07:41,083] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:07:41,084] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object third_fn at 0x7fe1bd34ca80, file "/tmp/ipykernel_171109/285673021.py", line 1> 
   2           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE
 

[2023-03-16 17:07:41,084] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/28567302

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

### a function that cannot be inlined

In [12]:
def third_fn(x: torch.Tensor):
    if x.sum() > 0:
        return x + 1
    return x

def main_fn(x: torch.Tensor, y:torch.Tensor):
    tmp = third_fn(x) # inline call here?
    return y + tmp

main_fn_1 = torch.compile(main_fn)

In [13]:
main_fn_1(x,x)

[2023-03-16 17:07:41,181] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn
[2023-03-16 17:07:41,182] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/2932769143.py:7
[2023-03-16 17:07:41,182] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL third_fn []
[2023-03-16 17:07:41,183] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:07:41,183] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:07:41,184] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object third_fn at 0x7fe1bb1cf0e0, file "/tmp/ipykernel_171109/2932769143.py", line 1> 
   2           0 LOAD_FAST                0 (x)
              2 LOAD_METHOD              0 (sum)
              4 CALL_METHOD              0
              6 LOAD_CONST               1 (0)
              8 COMPARE_OP               4 (>)
             10 POP_JUMP_IF_FALS

inline UserFunctionVariable() failed
__resume_at_6_6:
  7           0 LOAD_FAST                0 (___stack0)
              2 JUMP_ABSOLUTE           10
              4 LOAD_GLOBAL              0 (third_fn)
              6 LOAD_FAST                2 (x)
              8 CALL_FUNCTION            1
        >>   10 STORE_FAST               3 (tmp)

  8          12 LOAD_FAST                1 (y)
             14 LOAD_FAST                3 (tmp)
             16 BINARY_ADD
             18 RETURN_VALUE
__resume_at_12_8:
  2           0 JUMP_ABSOLUTE           14
              2 LOAD_FAST                0 (x)
              4 LOAD_ATTR                0 (sum)
              6 CALL_FUNCTION            0
              8 LOAD_CONST               1 (0)
             10 COMPARE_OP               4 (>)
             12 POP_JUMP_IF_FALSE       22

  3     >>   14 LOAD_FAST                0 (x)
             16 LOAD_CONST               2 (1)
             18 BINARY_ADD
             20 RETURN_VALUE

  4     >>   

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

In [14]:
def fn(a, b):
    return a.sum(0) + b.sum(0)

_dynamo.reset()
fn_ = torch.compile(fn)

a = torch.randn((33, 44), device='cuda')
b = torch.randn((33, 44), device='cuda')
fn_(a,b)
    

[2023-03-16 17:07:42,260] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing fn
[2023-03-16 17:07:42,261] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/1965302006.py:2
[2023-03-16 17:07:42,262] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST a []
[2023-03-16 17:07:42,262] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR sum [TensorVariable()]
[2023-03-16 17:07:42,263] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CONST 0 [GetAttrVariable(TensorVariable(), sum)]
[2023-03-16 17:07:42,264] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [GetAttrVariable(TensorVariable(), sum), ConstantVariable(int)]
[2023-03-16 17:07:42,266] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST b [TensorVariable()]
[2023-03-16 17:07:42,267] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR sum [TensorVariable(), TensorVariable()]
[2023-03-16 17:07:42,267] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CON

tensor([-10.5003,  -2.3082,   7.9250,   3.7306,  11.4371,   4.6230,   0.7037,
          8.5890,  11.2040,  -6.0124,  -3.6523,  -5.4520,  -7.1022,   1.9420,
         -0.8349,   0.2798,  -9.4945,  -7.4654,  -1.8288,  -9.7656,   1.7412,
         -0.9800,  -9.9994,   7.4591,  10.4829,   6.3766,   1.9650,  -6.3278,
         11.3239,   5.9450,  -1.2080, -11.4838,  -1.8048,   1.8252,   6.8341,
         -5.2005,   9.8097,  -4.1489,  -1.0129,   5.3399,  -4.0586,  -2.4990,
         -4.2183,  -3.4967], device='cuda:0')

## The dict, tuple can also be inlined

In [15]:
def third_fn1(x: torch.Tensor):
    adic = {}
    adic["out"] = x
    adic["tmp"] = x
    return adic

def main_fn1(x: torch.Tensor, y:torch.Tensor):
    tmp = third_fn1(x) # inline call here?
    return y + tmp["out"]

_dynamo.reset()
main_fn_2 = torch.compile(main_fn1)

In [16]:
a = torch.randn((33, 44), device='cuda')
b = torch.randn((33, 44), device='cuda')

In [17]:
main_fn_2(a, b)

[2023-03-16 17:07:42,661] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn1
[2023-03-16 17:07:42,661] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/2273668353.py:8
[2023-03-16 17:07:42,662] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL third_fn1 []
[2023-03-16 17:07:42,662] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:07:42,663] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:07:42,663] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object third_fn1 at 0x7fe07c0755b0, file "/tmp/ipykernel_171109/2273668353.py", line 1> 
   2           0 BUILD_MAP                0
              2 STORE_FAST               1 (adic)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_FAST                1 (adic)
              8 LOAD_CONST               1 ('out')
             10 STOR

tensor([[-3.7946, -0.5855,  1.0855,  ...,  0.1345, -0.1574,  2.2312],
        [ 1.3433,  2.8785, -1.6530,  ..., -1.7912, -2.0573,  0.7959],
        [-0.5695, -1.6292,  0.0336,  ..., -1.2363,  1.5298, -1.5409],
        ...,
        [ 0.2256,  0.0435,  1.8306,  ...,  1.2719,  0.9207,  0.6777],
        [-1.3494,  0.2264,  0.8916,  ..., -1.2846, -3.4267, -0.8233],
        [ 0.4636, -2.4870, -0.4616,  ..., -1.3939,  1.6246,  3.3484]],
       device='cuda:0')

In [18]:
def message(a, b):
    return a.view(-1, 1) + b.view(1, -1)

In [19]:
torch._dynamo.config.suppress_errors = True
def main_fn2(x: torch.Tensor, y:torch.Tensor):
    tmp = message(x, y) # inline call here?
    return tmp * 2

_dynamo.reset()
main_fn_2 = torch.compile(main_fn2)

In [20]:
main_fn_2(a, b)

[2023-03-16 17:07:43,103] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn2
[2023-03-16 17:07:43,104] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/1591432987.py:3
[2023-03-16 17:07:43,105] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL message []
[2023-03-16 17:07:43,106] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:07:43,106] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST y [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:07:43,107] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 2 [UserFunctionVariable(), TensorVariable(), TensorVariable()]
[2023-03-16 17:07:43,107] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object message at 0x7fe0ef421a80, file "/tmp/ipykernel_171109/2076900814.py", line 1> 
   2           0 LOAD_FAST                0 (a)
              2 LOAD_METHOD              0 (view)
              4 LOAD_CON

tensor([[-7.5893, -4.1648, -5.5210,  ..., -4.8884, -2.6454,  0.0884],
        [-4.5954, -1.1709, -2.5271,  ..., -1.8945,  0.3485,  3.0823],
        [ 0.1027,  3.5272,  2.1710,  ...,  2.8036,  5.0466,  7.7804],
        ...,
        [-5.4887, -2.0643, -3.4205,  ..., -2.7879, -0.5448,  2.1889],
        [-1.6947,  1.7298,  0.3736,  ...,  1.0062,  3.2492,  5.9830],
        [-0.9808,  2.4436,  1.0874,  ...,  1.7200,  3.9630,  6.6968]],
       device='cuda:0')

The inner-most graph break will cause the call stack failed to inline

In [23]:
def third_fn1(x: torch.Tensor):
    x = x + 1
    return x

def third_fn2(x: torch.Tensor):
    x = x + 1
    return x

def main_fn1(x: torch.Tensor, y:torch.Tensor):
    x = third_fn2(x) # inline call here?
    return y + x

_dynamo.reset()
main_fn_2 = torch.compile(main_fn1)

Both `third_fn1` and `third_fn2` could be inlined.

In [24]:
main_fn_2(a, b)

[2023-03-16 17:08:46,943] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn1
[2023-03-16 17:08:46,943] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/1078405638.py:10
[2023-03-16 17:08:46,944] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL third_fn2 []
[2023-03-16 17:08:46,945] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:08:46,945] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:08:46,946] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object third_fn2 at 0x7fe0f3748920, file "/tmp/ipykernel_171109/1078405638.py", line 5> 
   6           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 STORE_FAST               0 (x)

  7           8 LOAD_FAST                0 (x)
             10 RETURN_VALUE
 

[2023-03-16 1

tensor([[-2.7946,  0.4145,  2.0855,  ...,  1.1345,  0.8426,  3.2312],
        [ 2.3433,  3.8785, -0.6530,  ..., -0.7912, -1.0573,  1.7959],
        [ 0.4305, -0.6292,  1.0336,  ..., -0.2363,  2.5298, -0.5409],
        ...,
        [ 1.2256,  1.0435,  2.8306,  ...,  2.2719,  1.9207,  1.6777],
        [-0.3494,  1.2264,  1.8916,  ..., -0.2846, -2.4267,  0.1767],
        [ 1.4636, -1.4870,  0.5384,  ..., -0.3939,  2.6246,  4.3484]],
       device='cuda:0')

In [29]:
def third_fn1(x: torch.Tensor):
    x = x if x.sum() > 0 else x + 1 # force graph break
    return x

def third_fn2(x: torch.Tensor):
    x = third_fn1(x)
    return x

def main_fn1(x: torch.Tensor, y:torch.Tensor):
    x = third_fn2(x) # inline call here?
    return y + x

_dynamo.reset()
main_fn_2 = torch.compile(main_fn1)

In [30]:
main_fn_2(a, b)

[2023-03-16 17:11:57,040] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing main_fn1
[2023-03-16 17:11:57,040] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_171109/3616372052.py:10
[2023-03-16 17:11:57,041] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL third_fn2 []
[2023-03-16 17:11:57,042] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [UserFunctionVariable()]
[2023-03-16 17:11:57,042] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [UserFunctionVariable(), TensorVariable()]
[2023-03-16 17:11:57,043] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object third_fn2 at 0x7fe0f3589870, file "/tmp/ipykernel_171109/3616372052.py", line 5> 
   6           0 LOAD_GLOBAL              0 (third_fn1)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 STORE_FAST               0 (x)

  7           8 LOAD_FAST                0 (x)
             10 RETUR

inline UserFunctionVariable() failed
inline UserFunctionVariable() failed
__resume_at_6_19:
 10           0 LOAD_FAST                0 (___stack0)
              2 JUMP_ABSOLUTE           10
              4 LOAD_GLOBAL              0 (third_fn2)
              6 LOAD_FAST                2 (x)
              8 CALL_FUNCTION            1
        >>   10 STORE_FAST               2 (x)

 11          12 LOAD_FAST                1 (y)
             14 LOAD_FAST                2 (x)
             16 BINARY_ADD
             18 RETURN_VALUE
inline UserFunctionVariable() failed
__resume_at_6_20:
  6           0 LOAD_FAST                0 (___stack0)
              2 JUMP_ABSOLUTE           10
              4 LOAD_GLOBAL              0 (third_fn1)
              6 LOAD_FAST                1 (x)
              8 CALL_FUNCTION            1
        >>   10 STORE_FAST               1 (x)

  7          12 LOAD_FAST                1 (x)
             14 RETURN_VALUE


[2023-03-16 17:11:57,357] torch._inductor.graph: [INFO] Output code: /tmp/torchinductor_chunwei/hm/chmanlgj6f2724lhiyqptossmekp7giome7gykj4opjf3gqhxrjd.py
[2023-03-16 17:11:57,358] torch._inductor.compile_fx: [INFO] Step 3: torchinductor done compiling FORWARDS graph 14
[2023-03-16 17:11:57,359] torch._dynamo.output_graph: [INFO] Step 2: done compiler function debug_wrapper
[2023-03-16 17:11:57,360] torch._dynamo.output_graph: [INFO] TRACED GRAPH
 __compiled_fn_21 <eval_with_key>.89 opcode         name    target                  args        kwargs
-------------  ------  ----------------------  ----------  --------
placeholder    x       x                       ()          {}
call_method    sum_1   sum                     (x,)        {}
call_function  gt      <built-in function gt>  (sum_1, 0)  {}
output         output  output                  ((gt,),)    {}

[2023-03-16 17:11:57,361] torch._dynamo.symbolic_convert: [INFO] __resume_at_12_22 function
[2023-03-16 17:11:57,363] torch._dyna

__resume_at_12_22:
  2           0 JUMP_ABSOLUTE           14
              2 LOAD_FAST                0 (x)
              4 LOAD_ATTR                0 (sum)
              6 CALL_FUNCTION            0
              8 LOAD_CONST               1 (0)
             10 COMPARE_OP               4 (>)
             12 POP_JUMP_IF_FALSE       18
        >>   14 LOAD_FAST                0 (x)
             16 JUMP_FORWARD             6 (to 24)
        >>   18 LOAD_FAST                0 (x)
             20 LOAD_CONST               2 (1)
             22 BINARY_ADD
        >>   24 STORE_FAST               0 (x)

  3          26 LOAD_FAST                0 (x)
             28 RETURN_VALUE
__resume_at_16_23:
  2           0 JUMP_ABSOLUTE           18
              2 LOAD_FAST                0 (x)
              4 LOAD_ATTR                0 (sum)
              6 CALL_FUNCTION            0
              8 LOAD_CONST               1 (0)
             10 COMPARE_OP               4 (>)
             12 POP_JUMP

tensor([[-2.7946,  0.4145,  2.0855,  ...,  1.1345,  0.8426,  3.2312],
        [ 2.3433,  3.8785, -0.6530,  ..., -0.7912, -1.0573,  1.7959],
        [ 0.4305, -0.6292,  1.0336,  ..., -0.2363,  2.5298, -0.5409],
        ...,
        [ 1.2256,  1.0435,  2.8306,  ...,  2.2719,  1.9207,  1.6777],
        [-0.3494,  1.2264,  1.8916,  ..., -0.2846, -2.4267,  0.1767],
        [ 1.4636, -1.4870,  0.5384,  ..., -0.3939,  2.6246,  4.3484]],
       device='cuda:0')